In [77]:
import langchain

In [91]:
from langchain_community.llms.llamafile import Llamafile

llm = Llamafile(temperature=0)

# test
llm.invoke("What is Generative AI?")

'\nGenerative AI is a type of artificial intelligence that creates new content, ideas or solutions by learning from existing data. It uses algorithms and models to generate novel outputs, often in the form of text, images, music, videos or other forms of media. Generative AI can be used for various applications such as language translation, image recognition, natural language processing, and more. It is a powerful tool that has the potential to revolutionize many industries, including healthcare, finance, and education.</s>'

In [10]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader('https://arxiv.org/pdf/2402.07927')
pages = loader.load()


In [13]:
len(pages)


9

In [19]:
pages[0].page_content[0:500]

'A Systematic Survey of Prompt Engineering in Large Language Models:\nTechniques and Applications\nPranab Sahoo1,Ayush Kumar Singh1,Sriparna Saha1,Vinija Jain2,3,Samrat\nMondal1and Aman Chadha2,3\n1Department of Computer Science And Engineering, Indian Institute of Technology Patna\n2Stanford University,3Amazon AI\n{pranab_2021cs25, ayush_2211ai27, sriparna, samrat}@iitp.ac.in, hi@vinija.ai,\nhi@aman.ai\nAbstract\nPrompt engineering has emerged as an indispens-\nable technique for extending the capabilitie'

In [22]:
pages[0].metadata


{'source': 'https://arxiv.org/pdf/2402.07927', 'page': 0}

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)
splits = text_splitter.split_documents(pages)

In [27]:
len(splits)

40

In [30]:
splits[0]


Document(metadata={'source': 'https://arxiv.org/pdf/2402.07927', 'page': 0}, page_content='A Systematic Survey of Prompt Engineering in Large Language Models:\nTechniques and Applications\nPranab Sahoo1,Ayush Kumar Singh1,Sriparna Saha1,Vinija Jain2,3,Samrat\nMondal1and Aman Chadha2,3\n1Department of Computer Science And Engineering, Indian Institute of Technology Patna\n2Stanford University,3Amazon AI\n{pranab_2021cs25, ayush_2211ai27, sriparna, samrat}@iitp.ac.in, hi@vinija.ai,\nhi@aman.ai\nAbstract\nPrompt engineering has emerged as an indispens-\nable technique for extending the capabilities of large\nlanguage models (LLMs) and vision-language mod-\nels (VLMs). This approach leverages task-specific\ninstructions, known as prompts, to enhance model\nefficacy without modifying the core model param-\neters. Rather than updating the model parameters,\nprompts allow seamless integration of pre-trained\nmodels into downstream tasks by eliciting desired\nmodel behaviors solely based on th

In [93]:
from langchain_community.embeddings import LlamafileEmbeddings
embedding = LlamafileEmbeddings()

In [95]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"
sentence4 = "it is humid and hot"

embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)
embedding4 = embedding.embed_query(sentence4)

In [97]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    magnitude_vec1 = np.linalg.norm(vec1)
    magnitude_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (magnitude_vec1 * magnitude_vec2)
    print(f"Cosine Similarity: {similarity}")

In [99]:
cosine_similarity(embedding1, embedding2)
cosine_similarity(embedding2, embedding1)
cosine_similarity(embedding1, embedding3)
cosine_similarity(embedding3, embedding4)

Cosine Similarity: 0.7927743779555021
Cosine Similarity: 0.7927743779555021
Cosine Similarity: 0.458502299420121
Cosine Similarity: 0.5598645369516267


In [101]:
from langchain.vectorstores import Chroma
persist_directory = 'db/chroma/embeddingnew'

In [103]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [104]:
print(vectordb._collection.count())

40


In [107]:
question = "is there an email i can ask for help"
docs = vectordb.similarity_search(question,k=3)

In [109]:
for page in docs:
    print(f"found in page {page.metadata['page']}")

found in page 0
found in page 3
found in page 6


In [111]:
docs[0].page_content

"viding a structured overview of recent advancements\nin prompt engineering, categorized by application\narea. For each prompting approach, we provide a\nsummary detailing the prompting methodology, its\napplications, the models involved, and the datasets\nutilized. We also delve into the strengths and limita-\ntions of each approach and include a taxonomy dia-\ngram and table summarizing datasets, models, and\ncritical points of each prompting technique. This\nsystematic analysis enables a better understanding\nof this rapidly developing field and facilitates fu-\nture research by illuminating open challenges and\nopportunities for prompt engineering.\n1 Introduction\nPrompt engineering has emerged as a crucial technique for\nenhancing the capabilities of pre-trained large language mod-\nels (LLMs) and vision-language models (VLMs). It involves\nstrategically designing task-specific instructions, referred to as\nprompts, to guide model output without altering parameters.\nThe signific

In [113]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)


In [115]:
question = "What are major topics for this paper?"
result = qa_chain.invoke({"query": question})
result["result"]

' The paper discusses the use of language models in various fields, including natural language processing, computer vision, and robotics. It also explores the limitations and challenges associated with language models, such as their lack of contextual understanding and difficulty in handling out-of-vocabulary words. Additionally, the paper highlights recent advances in language model research, including transfer learning and multitasking capabilities.</s>'

In [70]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [73]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [75]:
question = "What are major topics for this paper?"
result = qa_chain.invoke({"query": question})
result["result"]

' The paper covers various topics related to language modeling, including language modeling basics, language modeling tasks, language modeling techniques, and language modeling research. It also discusses the challenges faced by LLMs in language modeling, such as the need for prompting and the importance of understanding human reasoning. The paper provides examples of how LLMs can be used to solve real-world problems, including reasoning, logic, and natural language generation.</s>'